In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import seaborn as sns
import os
import re

In [5]:
#os.chdir('../')
#os.chdir('Analysis')

os.getcwd()

'C:\\Users\\lgorman\\OneDrive\\Desktop\\PhD\\Analysis'

In [11]:
data=pd.read_csv(os.path.join('data',"processed","point_data_with_raster_values.csv")) # at this point we only preserve the grid geometry. Will need to add point geometry when reading the file back in again (same with country boundaries)  


In [12]:
world_bank_development_indicators=pd.read_csv(os.path.join("data","world_development_indicators","WDIData.csv"), low_memory=False)
world_bank_development_indicators_metadata=pd.read_csv(os.path.join("data","world_development_indicators","WDICountry.csv"), low_memory=False)

In [13]:
all_world_bank_countrys=world_bank_development_indicators_metadata["Country Code"]

In [14]:
# Three letter country code from RHoMIS
data["Three_Letter_Country_Code"]="NaN"
for row in data.index:
    world_bank_row_with_rhomis_two_letter_code=world_bank_development_indicators_metadata["2-alpha code"]==data.loc[row,"ID_COUNTRY"]
    data.loc[row,"Three_Letter_Country_Code"]=world_bank_development_indicators_metadata.loc[world_bank_row_with_rhomis_two_letter_code,"Country Code"].to_list()[0] #need the to list object to extract the object

In [15]:
# subsetting world bank indicators only for rhomis data countries
world_bank_development_indicators=world_bank_development_indicators.loc[world_bank_development_indicators["Country Code"].isin(data["Three_Letter_Country_Code"].unique()),:]

In [16]:
all_indicators=world_bank_development_indicators["Indicator Code"].unique()
all_indicators=["COUNTRY_LEVEL_"+indicator for indicator in all_indicators]
empty_matrix=np.zeros((data.shape[0],len(all_indicators)))
country_level_indicators=pd.DataFrame(empty_matrix, columns=all_indicators)
data_with_country_level_indicators=pd.concat([data,country_level_indicators],axis=1)

In [17]:
data_with_country_level_indicators=data_with_country_level_indicators.drop(columns="Unnamed: 0")
data_with_country_level_indicators.columns


Index(['ID_PROJ', 'ID_COUNTRY', 'YEAR', 'ITERATION', 'SURVEY_ID', 'ID_HH',
       'GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Country',
       ...
       'COUNTRY_LEVEL_SG.DMK.ALLD.FN.ZS', 'COUNTRY_LEVEL_SG.VAW.REAS.ZS',
       'COUNTRY_LEVEL_SG.VAW.ARGU.ZS', 'COUNTRY_LEVEL_SG.VAW.BURN.ZS',
       'COUNTRY_LEVEL_SG.VAW.GOES.ZS', 'COUNTRY_LEVEL_SG.VAW.NEGL.ZS',
       'COUNTRY_LEVEL_SG.VAW.REFU.ZS', 'COUNTRY_LEVEL_SP.M15.2024.FE.ZS',
       'COUNTRY_LEVEL_SP.M18.2024.FE.ZS', 'COUNTRY_LEVEL_SH.DYN.AIDS.FE.ZS'],
      dtype='object', length=1505)

In [48]:
def isNaN(num):
    return num != num

#---------------------Creating empty data frame of world bank indicators ------------------------#
test_string="COUNTRY_LEVEL_SG.DMK.ALLD.FN.ZS"
test_pattern='COUNTRY_LEVEL_'


pattern_for_country_level_variables='COUNTRY_LEVEL_'
all_indicators=world_bank_development_indicators["Indicator Code"].unique()
all_indicators=["COUNTRY_LEVEL_"+indicator for indicator in all_indicators]
empty_matrix=np.empty((len(data["YEAR"].unique())*len(data["Three_Letter_Country_Code"].unique()),(2+len(all_indicators))))
empty_matrix[:]=np.nan #filling the array with nan values to populate
column_names=["Three_Letter_Country_Code", "YEAR"]
[column_names.append(indicator) for indicator in all_indicators]
country_level_indicators=pd.DataFrame(empty_matrix, columns=column_names)
unique_countries=pd.Series(data["Three_Letter_Country_Code"].unique())
unique_year=pd.Series(data["YEAR"].unique())

#-------------------Filling data frame with potential year and country values -------------------#
for country_index in unique_countries.index:
    for year_index in unique_year.index:
        row_number_in_new= year_index+country_index*len(unique_year)
        country_level_indicators.loc[row_number_in_new,"Three_Letter_Country_Code"]=unique_countries[country_index]
        country_level_indicators.loc[row_number_in_new,"YEAR"]=int(unique_year[year_index])

#-------------------Filling data frame with potential year and country values -------------------#

for country in country_level_indicators["Three_Letter_Country_Code"].unique():
    #indicator_temp=world_bank_development_indicators.loc[world_bank_development_indicators["Country Code"]==country_temp,:]
    for year in country_level_indicators["YEAR"].unique():
        print (str(country)+str(int(year)))
        for column in all_indicators:
            if re.search(pattern_for_country_level_variables,column) is not None:
                temp_indicator=column.replace(test_pattern,'')
                subset1=country_level_indicators["Three_Letter_Country_Code"]==country 
                subset2=country_level_indicators["YEAR"]==year
                subset_world_bank_by_indicator=world_bank_development_indicators["Indicator Code"]==temp_indicator
                subset_world_bank_by_country=world_bank_development_indicators["Country Code"]==country
                
                year_temp=int(year)
                indicator_value_temp=float(world_bank_development_indicators.loc[subset_world_bank_by_indicator & subset_world_bank_by_country==True, str(int(year_temp))])
                if isNaN(indicator_value_temp)==False:
                    country_level_indicators.loc[subset1 & subset2==True,country_level_indicators.columns==column]=indicator_value_temp
                else:
                    while (isNaN(indicator_value_temp)==True):
                        year_temp=year_temp-1
                        if year_temp>2010: 
                            indicator_value_temp=float(world_bank_development_indicators.loc[subset_world_bank_by_indicator & subset_world_bank_by_country==True, str(int(year_temp))])
                            country_level_indicators.loc[subset1 & subset2==True,country_level_indicators.columns==column]=indicator_value_temp
                        else:
                            break

KEN2016
KEN2017
KEN2018
KEN2019
TZA2016
TZA2017
TZA2018
TZA2019
COD2016
COD2017
COD2018
COD2019
ZMB2016
ZMB2017
ZMB2018
ZMB2019
ETH2016
ETH2017
ETH2018
ETH2019
GHA2016
GHA2017
GHA2018
GHA2019
MLI2016
MLI2017
MLI2018
MLI2019
UGA2016
UGA2017
UGA2018
UGA2019
BDI2016
BDI2017
BDI2018
BDI2019
BFA2016
BFA2017
BFA2018
BFA2019
RWA2016
RWA2017
RWA2018
RWA2019


(44, 1431)

In [67]:
NA_indicators=pd.DataFrame(country_level_indicators.isna().any(axis=0)==False)
NA_indicators.columns=["No_Nas_in_Column"]
NA_indicators=list(NA_indicators[NA_indicators["No_Nas_in_Column"]==True].index)


country_level_indicators=country_level_indicators.loc[:,NA_indicators]
#country_level_indicators.loc[:,country_level_indicators.notna().any(axis=1)]
#[column for column in NA_indicators.columns]
#country_level_indicators.notna().any(axis=1)


In [68]:
data["country_year"]=data["Three_Letter_Country_Code"]+"_"+data["YEAR"].astype(str)

country_level_indicators["country_year"]=country_level_indicators["Three_Letter_Country_Code"]+"_"+country_level_indicators["YEAR"].astype(int).astype(str)

In [69]:
merge_wb_and_rhomis=pd.merge(data,country_level_indicators, on="country_year")
merge_wb_and_rhomis.to_csv(os.path.join("data", "processed", "point_data_processed.csv"))